In [16]:
!pip install gensim

In [18]:
import gensim
import re
import nltk
nltk.download('punkt')
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [19]:
# Train Word2Vec model on your comments

comments = [
    "I’d get a solid colored pink and maybe green or yellow pillows with it but that’s just me!",
    "Totally! solid and a few colors seems like the safest and best bet. buy two or three sets of sheets and mix/match",
    "100% this. Solid sheets, solid comforter",
    "I would try for cream coloured sheets and comforter (basic, no pattern) with some pink and lime/yellow accent pillows. ETA - love the mural by the way!",
    "I’m a big proponent of all white sheets (bed sheet, duvet cover, pillow cases), so you can bleach them while washing, and accent throw pillows and blanket(s) on top. I’d go with soft blue pink and yellow tones for those.",
    "Seconding this, you can also get white sheets and comforter with just a thin blue piping around, it’ll really tie the room together",
    "So far I’m liking the blue. Teal would be good too, chambray or mixed blues. Add an accent pillow if you like but keep most of the bedding a background color so focus remains on the mural.",
    "I was really hoping that was wallpaper because I LOVE it.",
    "Cream/terra cotta pink. I’d definitely not involve another pattern, the beautiful mural does so much work making the place special.",
    "I'd go with white/cream for sheets/duvet and accent with pillow colors + throw blanket color from the mural. I think going with a dark color for your bedspread would be very overwhelming.",
    "I personally would go with a darker blue to match one the blue colors of the mural or match a lighter blue from the mural. I think if you go with another color like yellow or white your eyes go to the bed first whereas I would want the mural to be my feature. I'd definitely add on colorful throw pillows though like the peachy orange, pink, yellow, etc so the bed still pops.",
    "I think a light blue solid linen duvet with a little fabric texture to match the walls without overpowering it. And then, maybe similar colored sheets and/or light pink or citron to switch it up ¯\_(ツ)_/¯ BEAUTIFUL mural though, you painted it yourself?!",
    "Yellow with darker pillows",
    "Cream or light pink",
    "I’d go with a plain pale yellow",
    "Move all the furniture out of the room so you can stare at the masterpiece all day Seriously though I dream of owning a home one day so I can paint fun murals on the walls.",
    "Omg that mural! 🥰🥰🥰🥰🥰",
    "Baby blue baby",
    "Clearly you love color, so embrace it! I’d pick one of the mural accent colors (pink, yellow, green, etc.) for a comforter, another for your sheets and a combo of all (with accents of blue) on throw pillows. You can have so much fun with this, and should! edit: comma",
    "You could really lean-in to the maximalism here. A vibrant large floral print would look amazing.",
    "The mural is quite marine so maybe like a raw linen green or earthy colour style to make the bed feel like your own island in the water. Or light blue to make it your 'safe bubble'. Pink would work quite nicely too, a nice dry sandy warm pink colour.",
    "I know this sounds weird but I'd try a Coffee berry color. Match the vibrancy in the color. Use a solid. Try not to match a color in the wallpaper."
]

# Preprocess comments
preprocessed_comments = [re.sub(r'[^\w\s]', '', comment.lower()) for comment in comments]

# Tokenize comments into words
comments_tokenized = [word_tokenize(comment) for comment in preprocessed_comments]

In [20]:
# Train Word2Vec model on your tokenized comments
model = gensim.models.Word2Vec(sentences=comments_tokenized, vector_size=100, window=5, min_count=1, workers=4)

# Get vectors for all tokenized comments
comment_vectors = [model.wv[comment] for comment in comments_tokenized]

# Save the Word2Vec model
model.save("word2vec_model")

In [21]:
# Load the model when needed
model = gensim.models.Word2Vec.load("word2vec_model")

In [35]:
import numpy as np

# Find the maximum length of comment vectors
max_length = max(len(comment_vector) for comment_vector in comment_vectors)

# Pad or truncate comment vectors to ensure they all have the same length
padded_comment_vectors = []
for comment_vector in comment_vectors:
    if len(comment_vector) < max_length:
        num_zeros = max_length - len(comment_vector)
        zero_array = np.zeros((num_zeros, comment_vector.shape[1]))
        padded_vector = np.concatenate((comment_vector, zero_array), axis=0)
    else:
        padded_vector = comment_vector[:max_length]
    padded_comment_vectors.append(padded_vector)

# Convert the list of padded comment vectors to a numpy array
X = np.array(padded_comment_vectors)

# Reshape comment vectors to flatten them
X_flat = X.reshape(X.shape[0], -1)

# Use K-means clustering
kmeans = KMeans(n_clusters=2)
kmeans.fit(X_flat)

# Assign each comment to a cluster
cluster_assignments = kmeans.labels_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [36]:
relevant_comments = [comments[i] for i, label in enumerate(cluster_assignments) if label == 0]
irrelevant_comments = [comments[i] for i, label in enumerate(cluster_assignments) if label == 1]

In [38]:
print("Relevant comment are:" ,relevant_comments)
print("Irrelevant comment are:", irrelevant_comments)

Relevant comment are: ['I’d get a solid colored pink and maybe green or yellow pillows with it but that’s just me!', 'Totally! solid and a few colors seems like the safest and best bet. buy two or three sets of sheets and mix/match', '100% this. Solid sheets, solid comforter', 'I would try for cream coloured sheets and comforter (basic, no pattern) with some pink and lime/yellow accent pillows. ETA - love the mural by the way!', 'I’m a big proponent of all white sheets (bed sheet, duvet cover, pillow cases), so you can bleach them while washing, and accent throw pillows and blanket(s) on top. I’d go with soft blue pink and yellow tones for those.', 'Seconding this, you can also get white sheets and comforter with just a thin blue piping around, it’ll really tie the room together', 'So far I’m liking the blue. Teal would be good too, chambray or mixed blues. Add an accent pillow if you like but keep most of the bedding a background color so focus remains on the mural.', 'I was really ho

In [40]:
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Combine relevant comments into a single text
relevant_text = ' '.join(relevant_comments)

# Tokenize the text into sentences
sentences = sent_tokenize(relevant_text)

# Use TF-IDF to compute sentence importance scores
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(sentences)
sentence_scores = np.sum(tfidf_matrix, axis=1)

# Flatten the array to make it 1-dimensional
sentence_scores = np.ravel(sentence_scores)

# Sort sentences by importance scores
sorted_indices = np.argsort(sentence_scores)[::-1]

# Select top sentences for the summary
num_sentences_in_summary = min(5, len(sorted_indices))
summary_indices = sorted_indices[:num_sentences_in_summary]
summary = [sentences[i] for i in summary_indices]

# Print the summary
print("Summary of Relevant Comments:")
print('\n'.join(summary))

Summary of Relevant Comments:
Yellow with darker pillows Cream or light pink I’d go with a plain pale yellow Move all the furniture out of the room so you can stare at the masterpiece all day Seriously though I dream of owning a home one day so I can paint fun murals on the walls.
I’m a big proponent of all white sheets (bed sheet, duvet cover, pillow cases), so you can bleach them while washing, and accent throw pillows and blanket(s) on top.
Seconding this, you can also get white sheets and comforter with just a thin blue piping around, it’ll really tie the room together So far I’m liking the blue.
The mural is quite marine so maybe like a raw linen green or earthy colour style to make the bed feel like your own island in the water.
Add an accent pillow if you like but keep most of the bedding a background color so focus remains on the mural.


In [45]:
!pip install bert-extractive-summarizer

In [48]:
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from summarizer import Summarizer

# Relevant comments
relevant_comments = [
    "Yellow with darker pillows Cream or light pink I’d go with a plain pale yellow Move all the furniture out of the room so you can stare at the masterpiece all day Seriously though I dream of owning a home one day so I can paint fun murals on the walls.",
    "I’m a big proponent of all white sheets (bed sheet, duvet cover, pillow cases), so you can bleach them while washing, and accent throw pillows and blanket(s) on top.",
    "Seconding this, you can also get white sheets and comforter with just a thin blue piping around, it’ll really tie the room together So far I’m liking the blue.",
    "The mural is quite marine so maybe like a raw linen green or earthy colour style to make the bed feel like your own island in the water.",
    "Add an accent pillow if you like but keep most of the bedding a background color so focus remains on the mural."
]

# Combine relevant comments into a single text
relevant_text = ' '.join(relevant_comments)

# Tokenize the text into sentences
sentences = sent_tokenize(relevant_text)

# Use BERT extractive summarizer to generate an extractive summary
summarizer = Summarizer()
extractive_summary = summarizer(relevant_text, ratio=0.2)  # You can adjust the ratio as needed

# Print the extractive summary
print("Extractive Summary:")
print(extractive_summary)

Extractive Summary:
Yellow with darker pillows Cream or light pink I’d go with a plain pale yellow Move all the furniture out of the room so you can stare at the masterpiece all day Seriously though I dream of owning a home one day so I can paint fun murals on the walls. The mural is quite marine so maybe like a raw linen green or earthy colour style to make the bed feel like your own island in the water.


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [49]:
# Approach 2 (no changes , it just removed the warning part)

import warnings
from nltk.tokenize import sent_tokenize
from summarizer import Summarizer

# Relevant comments
relevant_comments = [
    "Yellow with darker pillows Cream or light pink I’d go with a plain pale yellow Move all the furniture out of the room so you can stare at the masterpiece all day Seriously though I dream of owning a home one day so I can paint fun murals on the walls.",
    "I’m a big proponent of all white sheets (bed sheet, duvet cover, pillow cases), so you can bleach them while washing, and accent throw pillows and blanket(s) on top.",
    "Seconding this, you can also get white sheets and comforter with just a thin blue piping around, it’ll really tie the room together So far I’m liking the blue.",
    "The mural is quite marine so maybe like a raw linen green or earthy colour style to make the bed feel like your own island in the water.",
    "Add an accent pillow if you like but keep most of the bedding a background color so focus remains on the mural."
]

# Combine relevant comments into a single text
relevant_text = ' '.join(relevant_comments)

# Suppress sklearn warning
warnings.filterwarnings("ignore", category=FutureWarning)

# Use BERT extractive summarizer to generate an extractive summary
summarizer = Summarizer()
extractive_summary = summarizer(relevant_text, ratio=0.2)  # You can adjust the ratio as needed

# Print the extractive summary
print("Extractive Summary:")
print(extractive_summary)

Extractive Summary:
Yellow with darker pillows Cream or light pink I’d go with a plain pale yellow Move all the furniture out of the room so you can stare at the masterpiece all day Seriously though I dream of owning a home one day so I can paint fun murals on the walls. The mural is quite marine so maybe like a raw linen green or earthy colour style to make the bed feel like your own island in the water.
